<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Testando-Lógica" data-toc-modified-id="Testando-Lógica-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Testando Lógica</a></span></li></ul></div>

Ideias:

* Criar um diretório específico na pasta do projeto para alocar arquivos mp3 alvos do consumo do modelo
* Realizar gravações de áudio usando o app Voice Recorder e automaticamente posicionar os arquivos mp3 no diretório específico criado para leitura e consumo do modelo
* Criar script python com algumas regras de verificação de arquivos nesse diretório, como por exemplo:
    * Laço infinito de validação de presença de arquivos (ou então um start manual após a gravação)
    * Validação da presença de mais de 1 arquivo (warning pro usuário)
    * Validação de extensão do arquivo disponibilizado no diretório (necessariamente mp3)
* Após a leitura e consumo do modelo, propor a exclusão do arquivo a criação de um histórico de predições

In [1]:
# Importação de bibliotecas
import os
import librosa
import time

In [2]:
# Definindo variáveis de caminho
PROJECT_PATH = '/home/paninit/workspaces/voice-unlocker'
TARGET_PATH_NAME = 'audios_for_predict'
TARGET_PATH = os.path.join(PROJECT_PATH, TARGET_PATH_NAME)

AUDIO_EXT = '.mp3'

# Testando Lógica

In [3]:
# Criando diretório, caso inexistente
if not os.path.isdir(TARGET_PATH):
    os.makedirs(TARGET_PATH)

In [4]:
# Verificando arquivos no diretório
os.listdir(TARGET_PATH)

[]

In [5]:
# Regra 1 - quantidade de arquivos no diretório
valid_files = [file for file in os.listdir(TARGET_PATH) if os.path.splitext(file)[-1] == AUDIO_EXT]
qtd_files = len(valid_files)

if qtd_files > 1:
    print(f'Foram encontrados {qtd_files} arquivos {AUDIO_EXT} no diretório. Necessário validar um áudio por vez.')
qtd_files

0

___

In [6]:
# Regra 2 - nenhum arquivo encontrado com a extensão correta
if qtd_files == 0:
    print(f'Nenhum arquivo {AUDIO_EXT} encontrado no diretório. Verificar extensão do áudio disponibilizado.')

Nenhum arquivo .mp3 encontrado no diretório. Verificar extensão do áudio disponibilizado.


___

In [7]:
# Regra 3 - manter mais recente em caso de múltiplos audios
if qtd_files > 1:
    ctimes = [os.path.getctime(os.path.join(TARGET_PATH, file)) for file in valid_files]
    audio_ctimes = [time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ct)) for ct in ctimes] 
    print(audio_ctimes)

    idx_max_ctime = audio_ctimes.index(max(audio_ctimes))
    last_valid_audio = valid_files[idx_max_ctime]
    print(last_valid_audio)

___

In [8]:
# Regra 4 - expurgo dos áudios
for file in os.listdir(TARGET_PATH):
    os.remove(os.path.join(TARGET_PATH, file))